### Import Packages

In [4]:
import pandas as pd
import json
import numpy as np
import sys
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pickle

### Load Data

In [13]:
PE_rus = pd.read_csv('PowerExplorer -- RUS _ Russia.csv')

In [14]:
NewData = pd.read_csv('Rus_NewData_Translated.csv',index_col=0)

### Build the Matching Function

In [15]:
# Preprocess NewData
NewData['latitude'] = NewData['latitude'].astype(float)
NewData['longitude'] = NewData['longitude'].astype(float)
NewData['fuel_all_en'] = NewData[['fuel1_en','fuel2_en','fuel3_en','fuel4_en']].fillna('').apply(lambda x: ';'.join(x),axis = 1)
NewData = NewData.loc[(NewData['capacity_mw']!=0) | (NewData['name_en']=='Zheleznogorsk TPP'),:]

In [16]:
# Preprocess Database
PE_rus['Commissioning Date'] = PE_rus['Commissioning Date'].str.replace('2/27/17','2017')
PE_rus['Commissioning Date'] = [float(y[0])  if type(y)==list else np.NaN for y in PE_rus['Commissioning Date'].str.findall(r'\d{4}')]
PE_rus['fuel_all'] = PE_rus[['Fuel','Secondary Fuel']].fillna('').apply(lambda x: ';'.join(x),axis = 1)

In [17]:
# Help Functions
def score_string_similarity(string1,string2):
    return fuzz.token_set_ratio(string1,string2)

In [18]:
# Help Functions
def score_capacity(num1,num2):
    gap = np.abs(num1-num2)
    return gap/num1+gap/num2

In [19]:
# Help Functions
def score_euclidean_distance(lat1,lat2,lon1,lon2):  
    return np.sqrt((lat1-lat2)**2 + (lon1-lon2)**2)

In [20]:
def scoring_for_match(newdata_index,database_index):
    
    row_newdata = NewData.loc[newdata_index]
    row_database = PE_rus.loc[database_index]
    distance_score = score_euclidean_distance(row_newdata['latitude'],row_database['Latitude'],row_newdata['longitude'],row_database['Longitude'])
    name_similarity_score = score_string_similarity(row_newdata['name_en'],row_database['Name'])
    fuel_similarity_score = score_string_similarity(row_newdata['fuel_all_en'],row_database['fuel_all'])
    owner_similarity_score = score_string_similarity(row_newdata['owner_en'],row_database['Owner'])
    capacity_closeness_score = score_capacity(row_newdata['capacity_mw'],row_database['Capacity (MW)'])
    commissioning_year_score = np.abs(row_newdata['commissioning_year'] - row_database['Commissioning Date'])
    missing_fuel_score = 0
    if row_newdata['fuel_all_en']==';;;' or row_database['fuel_all']==';;;':
        missing_fuel_score = 1
    
    match_dict = {'new_index':newdata_index,
                  'old_index':database_index,
                  'distance_score':distance_score,
                  'name_similarity_score':name_similarity_score,
                  'fuel_similarity_score':fuel_similarity_score,
                  'capacity_closeness_score':capacity_closeness_score,
                  'owner_similarity_score':owner_similarity_score,
                  'commissioning_year_score':commissioning_year_score,
                  'missing_fuel_score':missing_fuel_score}
    
    return match_dict
    

In [21]:
def scoring_for_single_example(example_id):
    scoring_df = pd.DataFrame(columns = ['capacity_closeness_score',
                                         'distance_score',
                                         'fuel_similarity_score',
                                         'name_similarity_score',
                                         'owner_similarity_score',
                                         'commissioning_year_score',
                                         'missing_fuel_score',
                                         'new_index',
                                         'old_index'])
    for i in range(PE_rus.shape[0]):
        scoring_df = scoring_df.append(scoring_for_match(example_id,i),ignore_index=True)
    return scoring_df

In [22]:
def result_sorting(column):  
    if column == 'name':
        return scoring_df.sort_values(by = ['name_similarity_score','distance_score'], ascending = False).head()
    if column == 'distance':
        return scoring_df.sort_values(by = 'distance_score', ascending = True).head()
    if column == 'fuel':
        return scoring_df.sort_values(by = 'fuel_similarity_score', ascending = False).head()
    if column == 'capacity':
        return scoring_df.sort_values(by = 'capacity_closeness_score', ascending = True).head()
    if column == 'owner':
        return scoring_df.sort_values(by = 'owner_similarity_score', ascending = False).head()
    if column == 'commissioning_year':
        return scoring_df.sort_values(by = 'commissioning_year_score', ascending = True).head()

In [23]:
def manual_check(newdata_index,database_index):
    print('NewData:\n')
    print(NewData.loc[newdata_index,['name_en','fuel_all_en','capacity_mw','latitude','longitude','owner_en','commissioning_year']])
    print('\nDataBase:\n')
    print(PE_rus.loc[database_index,['Name','fuel_all','Capacity (MW)','Latitude','Longitude','Owner','Commissioning Date']])

In [24]:
def update_matching_index(df, new_index, matched = False, old_index = None):
    if matched and old_index != None:
        row = {'new_index':new_index, 'found_match':True, 'old_index':old_index}
    elif matched and old_index == None:
        print('Provide the old index if there is a match')
        return
    else:
        row = {'new_index':new_index, 'found_match':False, 'old_index':'NA'}
    df = df.append(row,ignore_index=True)
    return df

### Develop the Matching Table by Manually Check

In [25]:
# Initialize an empty data frame to store the manual check results
matching_index_df = pd.DataFrame(columns = ['new_index','found_match','old_index'])

In [26]:
#Get the scores for a single data point
scoring_df = scoring_for_single_example(1)

In [27]:
#Sort out the matches with most similar names
result_sorting('name')

,capacity_closeness_score,distance_score,fuel_similarity_score,name_similarity_score,owner_similarity_score,commissioning_year_score,missing_fuel_score,new_index,old_index
135,9.552172,11.679101,100.0,53.0,29.0,14.0,0.0,1.0,135.0
122,0.144427,0.004390,100.0,51.0,78.0,0.0,0.0,1.0,122.0
70,1.846423,48.849688,27.0,49.0,52.0,44.0,0.0,1.0,70.0
99,2.154023,0.838054,100.0,48.0,33.0,NaN,0.0,1.0,99.0
119,1.636503,51.563221,27.0,47.0,52.0,47.0,0.0,1.0,119.0


In [28]:
#Sort out the matches with closest distance
result_sorting('distance')

,capacity_closeness_score,distance_score,fuel_similarity_score,name_similarity_score,owner_similarity_score,commissioning_year_score,missing_fuel_score,new_index,old_index
122,0.144427,0.004390,100.0,51.0,78.0,0.0,0.0,1.0,122.0
99,2.154023,0.838054,100.0,48.0,33.0,NaN,0.0,1.0,99.0
241,1.546518,0.842028,12.0,34.0,27.0,NaN,0.0,1.0,241.0
87,1.774490,0.997013,27.0,6.0,78.0,52.0,0.0,1.0,87.0
71,1.224577,1.060497,100.0,29.0,78.0,22.0,0.0,1.0,71.0


In [29]:
#Sort out the matches with closest commissioning year
result_sorting('commissioning_year')

,capacity_closeness_score,distance_score,fuel_similarity_score,name_similarity_score,owner_similarity_score,commissioning_year_score,missing_fuel_score,new_index,old_index
122,0.144427,0.004390,100.0,51.0,78.0,0.0,0.0,1.0,122.0
110,0.243318,77.080918,27.0,31.0,35.0,8.0,0.0,1.0,110.0
135,9.552172,11.679101,100.0,53.0,29.0,14.0,0.0,1.0,135.0
174,8.058314,8.437779,100.0,23.0,29.0,16.0,0.0,1.0,174.0
75,0.053611,1.097956,100.0,6.0,78.0,16.0,0.0,1.0,75.0


In [30]:
#Sort out the matches with closest capacity
result_sorting('capacity')

,capacity_closeness_score,distance_score,fuel_similarity_score,name_similarity_score,owner_similarity_score,commissioning_year_score,missing_fuel_score,new_index,old_index
237,0.023630,1.692251,100.0,25.0,29.0,48.0,0.0,1.0,237.0
183,0.030797,4.748946,100.0,25.0,29.0,91.0,0.0,1.0,183.0
243,0.037009,101.977340,100.0,31.0,27.0,NaN,0.0,1.0,243.0
208,0.044641,22.716897,100.0,28.0,20.0,82.0,0.0,1.0,208.0
76,0.044641,1.161026,100.0,36.0,78.0,17.0,0.0,1.0,76.0


In [31]:
# Print out and check the pairs that you feel most likely to be a match
manual_check(304,57)

NewData:

name_en                     Zakamskaya CHP Plant-5
fuel_all_en                                    ;;;
capacity_mw                                     69
latitude                                   58.0772
longitude                                  55.7706
owner_en              Perm Branch of PJSC "T Plus"
commissioning_year                            1936
Name: 304, dtype: object

DataBase:

Name                    Zakamskaya
fuel_all              Natural Gas;
Capacity (MW)                   69
Latitude                       NaN
Longitude                      NaN
Owner                      t+group
Commissioning Date            1986
Name: 57, dtype: object


In [4396]:
# Store the result if there is a match
matching_index_df = update_matching_index(matching_index_df,300,True,148)

In [4372]:
# Store the result if there is not matches
matching_index_df = update_matching_index(matching_index_df,298,False)

In [15]:
# Save the matching table
matching_index_df.to_csv('matching_index_df_v2.csv')

### Derive Training Data from Matching Table

In [321]:
matching_index_df = pd.read_csv('matching_index_df_v2.csv',index_col= 0)

In [322]:
all_data = pd.DataFrame(columns = ['capacity_closeness_score',
                                   'distance_score',
                                   'fuel_similarity_score',
                                   'name_similarity_score',
                                   'commissioning_year_score',
                                   'owner_similarity_score',
                                   'missing_fuel_score',
                                   'new_index',
                                   'old_index',
                                   'match'])

for i in matching_index_df['new_index']:
    temp_data = scoring_for_single_example(i)
    top_distance = temp_data.sort_values(by = 'distance_score',ascending = True).head(3)
    top_name = temp_data.sort_values(by = 'name_similarity_score',ascending = False).head(3)
    top_capacity = temp_data.sort_values(by = 'capacity_closeness_score',ascending = True).head(3)
    top_commissioning_year = temp_data.sort_values(by = 'commissioning_year_score',ascending = True).head(3)
    #top_fuel_type = temp_data.sort_values(by = 'fuel_similarity_score',ascending = False).head(2)
    #top_owner = temp_data.sort_values(by = 'owner_similarity_score',ascending = False).head(2)
    all_data = pd.concat([all_data,top_distance,top_name,top_capacity,top_commissioning_year],axis = 0)
    if i % 10 == 0:
        print(i,'done!')

0 done!
10 done!
20 done!
30 done!
40 done!
50 done!
60 done!
70 done!
80 done!
90 done!
100 done!
110 done!
120 done!
130 done!
140 done!
150 done!
160 done!
170 done!
180 done!
190 done!
210 done!
230 done!
240 done!
250 done!
260 done!
290 done!
300 done!


In [324]:
all_data.loc[:,'match'] = 0

In [325]:
all_data = pd.merge(all_data,matching_index_df,how = 'left',on = ['new_index','old_index'])

In [327]:
all_data.loc[all_data['found_match']==True,'match'] = 1

In [332]:
all_data = all_data.drop(['found_match'],axis = 1)

In [333]:
all_data['na_capacity_score'] = 0 
all_data.loc[all_data['capacity_closeness_score'].isnull(),'na_capacity_score'] = 1
all_data['na_distance_score'] = 0 
all_data.loc[all_data['distance_score'].isnull(),'na_distance_score'] = 1
all_data['na_commissioning_score'] = 0 
all_data.loc[all_data['commissioning_year_score'].isnull(),'na_commissioning_score'] = 1

In [334]:
all_data = all_data.drop_duplicates()

In [336]:
# Save the training set
all_data.to_csv('all_data_262(matching_top3_10feature).csv')

### Train Random Forest Model

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import confusion_matrix
from itertools import combinations

C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [337]:
origin_data = pd.read_csv('all_data_262(matching_top3_10feature).csv',index_col=0)

In [341]:
all_data = origin_data.drop(['new_index','old_index'],axis = 1)
all_data.loc[all_data['missing_fuel_score']==1,'fuel_similarity_score'] = 50
all_data.loc[:,'distance_score'] = all_data['distance_score'].fillna(all_data['distance_score'].mean())
all_data.loc[:,'capacity_closeness_score'] = all_data['capacity_closeness_score'].fillna(all_data['capacity_closeness_score'].mean())
all_data.loc[:,'commissioning_year_score'] = all_data['commissioning_year_score'].fillna(all_data['commissioning_year_score'].mean())
#all_data.loc[:,'fuel_similarity_score'] = 100-all_data['fuel_similarity_score']
#all_data.loc[:,'name_similarity_score'] = 100-all_data['name_similarity_score']
#all_data.loc[all_data['capacity_closeness_score']==np.inf,'capacity_closeness_score'] = 10000

In [344]:
y_data = all_data['match']
X_data = all_data.drop(['match'],axis = 1)

In [396]:
# Cross Validation Function
def cv_stratified_fit_and_score(model,cv):
    skf = StratifiedKFold(y_data,n_folds=cv,random_state=0) 
    count = 0
    cv_scores_val = []
    cv_scores_train = []
    wrong_indexes = None
    wrong_indexes_acc = []
    wrong_cases = {}
    
    for train_index,val_index in skf:
        count+=1
        X_train,X_valid = X_data.iloc[train_index],X_data.iloc[val_index]
        y_train,y_valid = y_data.iloc[train_index],y_data.iloc[val_index]
        
#         Scaler = MinMaxScaler(feature_range=(0,100))
#         X_train = Scaler.fit_transform(X_train)
#         X_valid = Scaler.transform(X_valid)
        
        model.fit(X_train,y_train)
        #y_pred = model.predict_proba(X_valid)
        cv_scores_val.append(model.score(X_valid,y_valid))
        cv_scores_train.append(model.score(X_train,y_train))
        
        wrong_indexes = list(y_valid[model.predict(X_valid)!=y_valid].index)
        wrong_indexes_acc += wrong_indexes
        
#         if len(wrong_indexes)!=0:
#             wrong_pro = model.predict_proba(X_valid.loc[wrong_indexes])
#             for i in range(len(wrong_pro)):
#                 wrong_cases[wrong_indexes[i]] = wrong_pro[i]
                
        print('fold {}, done.'.format(count))
    return cv_scores_val,cv_scores_train,wrong_indexes_acc,wrong_cases

In [474]:
# The parameters are tuned here
rf = RandomForestClassifier(n_estimators=200,random_state=0,max_features=3)

In [473]:
results = cv_stratified_fit_and_score(rf,5)
validation_mean = np.mean(results[0])
train_mean = np.mean(results[1])
print('Validation Accuracy: {}'.format(validation_mean),'\nTraining Accuracy: {}'.format(train_mean))

fold 1, done.
fold 2, done.
fold 3, done.
fold 4, done.
fold 5, done.
Validation Accuracy: 0.9892958820538892 
Training Accuracy: 0.9996435731258423


In [224]:
rf_model = rf.fit(X_data,y_data)

In [476]:
# Save the model for future prediction
filename = 'rf_model_v1.sav'
pickle.dump(rf_model, open(filename, 'wb'))

### Predict Matches for Unseen Data

In [482]:
filename = 'rf_model_v1.sav'

In [483]:
# Load the model saved previously
rf_model = pickle.load(open(filename, 'rb'))

In [484]:
test_data = NewData.loc[301:,:]

In [672]:
def predict_on_single_test(model,new_index):
    all_data = pd.DataFrame(columns = ['capacity_closeness_score',
                                       'distance_score',
                                       'fuel_similarity_score',
                                       'name_similarity_score',
                                       'commissioning_year_score',
                                       'owner_similarity_score',
                                       'missing_fuel_score',
                                       'new_index',
                                       'old_index',
                                       'match'])
    temp_data = scoring_for_single_example(new_index)
    top_distance = temp_data.sort_values(by = 'distance_score',ascending = True).head(3)
    top_name = temp_data.sort_values(by = 'name_similarity_score',ascending = False).head(3)
    top_capacity = temp_data.sort_values(by = 'capacity_closeness_score',ascending = True).head(3)
    top_commissioning_year = temp_data.sort_values(by = 'commissioning_year_score',ascending = True).head(3)
    all_data = pd.concat([all_data,top_distance,top_name,top_capacity,top_commissioning_year],axis = 0)
    all_data = all_data.drop(['new_index','old_index'],axis = 1)
    all_data['na_capacity_score'] = 0 
    all_data.loc[all_data['capacity_closeness_score'].isnull(),'na_capacity_score'] = 1
    all_data['na_distance_score'] = 0 
    all_data.loc[all_data['distance_score'].isnull(),'na_distance_score'] = 1
    all_data['na_commissioning_score'] = 0 
    all_data.loc[all_data['commissioning_year_score'].isnull(),'na_commissioning_score'] = 1
    all_data.loc[all_data['missing_fuel_score']==1,'fuel_similarity_score'] = 50
    all_data.loc[:,'distance_score'] = all_data['distance_score'].fillna(origin_data['distance_score'].mean())
    all_data.loc[:,'capacity_closeness_score'] = all_data['capacity_closeness_score'].fillna(origin_data['capacity_closeness_score'].mean())
    all_data.loc[:,'commissioning_year_score'] = all_data['commissioning_year_score'].fillna(origin_data['commissioning_year_score'].mean())
    all_data.drop_duplicates(inplace=True)
    X_test = all_data.drop(['match'],axis = 1)
    prediction = model.predict(X_test)
    prediction_proba = model.predict_proba(X_test)
    return prediction,prediction_proba,all_data

In [673]:
test_index = list(test_data.index)

In [1261]:
while (len(test_index)!=0):
    
    i = test_index[0]
    print('Predicting on index {}'.format(i))
    
    try:
        results = predict_on_single_test(rf_model,i)
        proba_0,proba_1 = results[1][:,0],results[1][:,1]
        if np.all(proba_0 > 0.9):
            matching_index_df = update_matching_index(matching_index_df,i,False)
        elif np.any(proba_1 > 0.8):
            if (proba_1 > 0.8).sum() > 1:
                print('Found multiple matches for index {}, need manually check.'.format(i))
                test_index.pop(0)
                break
            else:
                old_index = results[2].index[np.argmax(proba_1)]
                matching_index_df = update_matching_index(matching_index_df,i,True,old_index)
        else:
            print('The model is not pretty sure for index {}, need manually check.'.format(i))
            test_index.pop(0)
            break
            
    except:
        print('{} is not in the list.'.format(i))
        
    test_index.pop(0)

Predicting on index 256
Predicting on index 403
The model is not pretty sure for index 403, need manually check.


In [1315]:
manual_check(476,184)

NewData:

name_en                           Leningrad NPP-2
fuel_all_en                                   ;;;
capacity_mw                                  4340
latitude                                  59.8304
longitude                                 29.0567
owner_en              JSC "Concern Rosenergoatom"
commissioning_year                            NaN
Name: 476, dtype: object

DataBase:

Name                  Leningrad NPP
fuel_all                   Nuclear;
Capacity (MW)                  4000
Latitude                    59.8401
Longitude                   29.0331
Owner                       Rosatom
Commissioning Date             1973
Name: 184, dtype: object


In [1265]:

matching_index_df = update_matching_index(matching_index_df,403,True,42)

In [1257]:
matching_index_df = update_matching_index(matching_index_df,935,False)

In [ ]:
matching_index_df.to_csv('prediction_matching_table.csv')

### Labeling flags

In [67]:
matching_table = pd.read_csv('prediction_matching_table.csv',index_col=0)

In [ ]:
matched = matching_table[matching_table['found_match']==1]

In [ ]:
# new_data_features = ['name_en','latitude','longitude','owner_en','commissioning_year','fuel_all_en','capacity_mw']
# data_base_feature = ['Name','Latitude','Longitude','Owner','Commissioning Date','fuel_all','Capacity (MW)']
# columns = ['database_index','matched_index','database_name','matched_name','name_similarity_score',
#           'database_latitude','matched_latitude','database_longitude','matched_longitude','distance_score',
#           'database_owner','matched_owner','owner_similarity_score',
#           'database_commissioning_year','matched_commissioning_year','commissioning_year_score',
#           'database_fuel','matched_fuel','fuel_similarity_score',
#           'database_capacity','matched_capacity','capacity_closeness_score','missing_fuel_score']

In [109]:
columns = ['name_en','Name','name_similarity_score','latitude','Latitude','longitude','Longitude','distance_score','owner_en','Owner','owner_similarity_score','commissioning_year','Commissioning Date','commissioning_year_score','fuel_all_en','fuel_all','fuel_similarity_score','capacity_mw','Capacity (MW)','capacity_closeness_score']

In [110]:
flaged_matches = pd.DataFrame(columns = columns)

In [144]:
for i in list(matched.index):
    new_index = matched.loc[i,'new_index']
    old_index = matched.loc[i,'old_index']
    x = NewData.loc[new_index,new_data_features].to_dict()
    y = PE_rus.loc[old_index,data_base_feature].to_dict()
    x.update(y)
    x.update(scoring_for_match(new_index,old_index))
    flaged_matches = flaged_matches.append(x,ignore_index=True)

In [151]:
flaged_matches.rename(columns={'name_en':'matched_name','Name':'database_name','latitude':'matched_latitude','Latitude':'database_latitude','longitude':'matched_longitude','Longitude':'database_longitude','owner_en':'matched_owner','Owner':'database_owner','commissioning_year':'matched_commissioning_year','Commissioning Date':'database_commissioning_year','fuel_all_en':'matched_fuel_type','fuel_all':'database_fuel_type','capacity_mw':'matched_capacity','Capacity (MW)':'database_capacity'}, inplace=True)

In [154]:
flaged_matches.to_csv('flagged_matching_table.csv')